In [1]:
!pip install pdfplumber

In [2]:
import pdfplumber
import re

def extract_text_from_pdf(pdf_path):
    """
    Extracts all text from a PDF file.
    Args:
        pdf_path (str): The path to the PDF file.
    Returns:
        str: The concatenated text from all pages of the PDF.
    """
    full_text = ''
    try:
        with pdfplumber.open(pdf_path) as pdf:
            print(f'The document ({pdf_path}), has {pdf.pages} pages.')
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text: 
                    full_text += page_text + "\n" 
    except Exception as e:
        print(f'Error extracting text from {pdf_path}: {e}')
        return None
    return full_text

pdf_file = "./GoldmanSachs__2024-annual-report.pdf"  
extracted_content = extract_text_from_pdf(pdf_file)

if extracted_content:
    print("Text successfully extracted for NLP analysis.")
    print(extracted_content[:500])
else:
    print("Failed to extract text.")

The document (./GoldmanSachs__2024-annual-report.pdf), has [<Page:1>, <Page:2>, <Page:3>, <Page:4>, <Page:5>, <Page:6>, <Page:7>, <Page:8>, <Page:9>, <Page:10>, <Page:11>, <Page:12>, <Page:13>, <Page:14>, <Page:15>, <Page:16>, <Page:17>, <Page:18>, <Page:19>, <Page:20>, <Page:21>, <Page:22>, <Page:23>, <Page:24>, <Page:25>, <Page:26>, <Page:27>, <Page:28>, <Page:29>, <Page:30>, <Page:31>, <Page:32>, <Page:33>, <Page:34>, <Page:35>, <Page:36>, <Page:37>, <Page:38>, <Page:39>, <Page:40>, <Page:41>, <Page:42>, <Page:43>, <Page:44>, <Page:45>, <Page:46>, <Page:47>, <Page:48>, <Page:49>, <Page:50>, <Page:51>, <Page:52>, <Page:53>, <Page:54>, <Page:55>, <Page:56>, <Page:57>, <Page:58>, <Page:59>, <Page:60>, <Page:61>, <Page:62>, <Page:63>, <Page:64>, <Page:65>, <Page:66>, <Page:67>, <Page:68>, <Page:69>, <Page:70>, <Page:71>, <Page:72>, <Page:73>, <Page:74>, <Page:75>, <Page:76>, <Page:77>, <Page:78>, <Page:79>, <Page:80>, <Page:81>, <Page:82>, <Page:83>, <Page:84>, <Page:85>, <Page:86>, <Pa

In [3]:
import requests

OLLAMA_URL = "http://localhost:11434"
def chat_with_llama3(prompt, model="llama3.2", stream=False):
    """
    Send a prompt to Llama3 and get response
    
    Args:
        prompt (str): The prompt to send
        model (str): Model name (default: llama3)
        stream (bool): Whether to stream response
    
    Returns:
        str: The response from the model
    """
    url = f"{OLLAMA_URL}/api/generate"
    
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": stream
    }
    
    try:
        response = requests.post(url, json=payload)
        
        if response.status_code == 200:
            if stream:
                # Handle streaming response
                full_response = ""
                for line in response.iter_lines():
                    if line:
                        chunk = json.loads(line.decode('utf-8'))
                        if 'response' in chunk:
                            print(chunk['response'], end='', flush=True)
                            full_response += chunk['response']
                        if chunk.get('done', False):
                            break
                print()  # New line at the end
                return full_response
            else:
                # Handle non-streaming response
                result = response.json()
                return result.get('response', 'No response received')
        else:
            return f"Error: {response.status_code} - {response.text}"
            
    except Exception as e:
        return f"Error: {e}"

# Test the function
response = chat_with_llama3("Hello! Can you tell me a joke?")
print("Response:", response)

Response: Here's one:

What do you call a fake noodle?

(Wait for it...)

An impasta!

Hope that made you laugh! Do you want to hear another one?


In [4]:
def chunk_texts(root_text: str, chunk_size:int = 10, overlap:int = 0, split_on_whitespace_only=True) -> list[str]:
    chunks: list[str] = list()
    index: int = 0
    len_whole_text: int = len(root_text)
    if len_whole_text == 0:
        raise Exception('No Root text sent for chunking into chunk_texts()')
    while index < len_whole_text:
        if split_on_whitespace_only:
            prev_whitespace = 0
            left_index = index - overlap
            while left_index >= 0:
                if root_text[left_index] == " ":
                    prev_whitespace = left_index
                    break
                left_index -= 1
            next_whitespace = root_text.find(" ", index + chunk_size)
            if next_whitespace == -1:
                next_whitespace = len_whole_text
            chunks.append(root_text[prev_whitespace:next_whitespace].strip())
            index = next_whitespace + 1
        else:
            start = max(0, index - overlap + 1)
            end = min(index + chunk_size + overlap, len_whole_text)
            chunks.append(root_text[start:end].strip())
            index += chunk_size
    return chunks

chunks = chunk_texts(extracted_content, chunk_size = 50)
print(len(chunks))

16617


In [23]:
#!pip uninstall keras

^C


In [25]:
#!pip uninstall tf-keras

^C


A wide range of models are available on the Hugging Face Model Hub. Popular choices include:
all-MiniLM-L6-v2: Produces 384-dimensional embeddings. It offers a good balance between performance and speed.
all-mpnet-base-v2: Generates 768-dimensional embeddings and is a high-performing base model for semantic search.
mixedbread-ai/mxbai-embed-large-v1: A high-performance model that produces 1024-dimensional embeddings. 

In [12]:
#note: keras 3 is incompatibel with sentence_gransformers and raised an exception
#pip install tf-keras was executed in response
from sentence_transformers import SentenceTransformer

# Load a pre-trained model
#model = SentenceTransformer("all-MiniLM-L6-v2")
model = SentenceTransformer("all-mpnet-base-v2")
embeddings = model.encode(chunks)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\tonmo\Development\AI\env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tonmo\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
print(f'Num of Embeddings is {len(embeddings)}, with the number of dims for first embeddins being {len(embeddings[0])}')

Num of Embeddings is 16617, with the number of dims for first embeddins being 768


In [14]:
!pip install neo4j

In [15]:
from neo4j import GraphDatabase

# Connect to Neo4j over Bolt
uri = "bolt://localhost:7687"
username = "neo4j"
password = "password1"

driver = GraphDatabase.driver(uri, auth=(username, password))


In [16]:
driver